In [1]:
import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

DEBUG = False
EPOCHS = 10
HF_USERNAME = None
from huggingface_hub import HfApi
api = HfApi()
if HF_USERNAME is None:
    HF_USERNAME = api.whoami()["name"]

In [2]:
import config

In [3]:
def upload(filename, tensor, dataset):
    assert ".pt" in filename
    path = HF_USERNAME + "/" + dataset

    torch.save(tensor, filename)
    if not DEBUG:
        api.upload_file(
                path_or_fileobj="./" + filename,
                path_in_repo=filename,
                repo_id= dataset,
                repo_type="dataset"
            )
    else:
        print("skipping upload of", filename, tensor)
    os.remove(filename)

def split_into_epochs(l):
    return torch.tensor_split(torch.tensor(l), EPOCHS)
def repeat_for_ten_epochs(l):
    return [torch.tensor(l) for _ in range(0,10)]

In [4]:
from baseline_curricula import get_perplexity_ngram

for dataset_name in config.datasets:
    perplexity_df = get_perplexity_ngram(dataset_name)
    upload("perplexity_increasing.pt", repeat_for_ten_epochs(perplexity_df["perplexity"].sort_values(ascending=True).index.to_list()), dataset_name)


./difficulty_ngram/1/babylm_2024_10m_curriculum
Skipping ./difficulty_ngram/1/babylm_2024_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


In [5]:
from baseline_curricula import get_mattr


for dataset_name in config.datasets:
    mattr_df = get_mattr(dataset_name, n_proc=128)
 #   upload("perplexity_decreasing.pt", split_into_epochs(perplexity_df["perplexity"].sort_values(ascending=False).index.to_list()), dataset)
    upload("mattr_increasing.pt", repeat_for_ten_epochs(mattr_df["mattr"].sort_values(ascending=True).index.to_list()), dataset_name)


./difficulty_mattr/babylm_2024_10m_curriculum
Skipping ./difficulty_mattr/babylm_2024_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


In [6]:
mattr_df

,mattr
document_id,
0,1.000000
1,1.000000
2,1.000000
3,1.000000
4,1.000000
...,...
1179009,0.995238
1179010,0.993007
1179011,0.981818


In [7]:
from baseline_curricula import get_mattr
import torch


for dataset_name in config.datasets:
    
    torch.manual_seed(42)
    dataset = load_dataset(dataset_name)
    random_order = [torch.randperm(len(dataset["train"])) for _ in range(0,EPOCHS)]
    
    upload("random.pt", random_order, dataset_name)


No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
# verify compliance with babylm 10*10M track


from util import get_curriculum
from itertools import product


word_count = lambda d: {"words" : len(d["text"].split())}

def validate_training_duration_limitation(dataset_name, curriculum_name):
    dataset = load_dataset(dataset_name)
    curriculum = torch.stack(get_curriculum(dataset_name, curriculum_name)).flatten()

    word_counts = dataset["train"].map(word_count, num_proc=100)
    training_data = word_counts.select(curriculum)
    total_words_seen = sum(training_data["words"])
    print(f"[{dataset_name} -> {curriculum_name}] {total_words_seen} words : {len(training_data)} documents")
    assert total_words_seen <= 10*10000000
    assert total_words_seen >= 9*10000000
    return True

[validate_training_duration_limitation(dataset_name, curriculum_name) for dataset_name, curriculum_name in product(config.datasets, config.curricula)]


In [9]:
from concurrent.futures import ProcessPoolExecutor
from itertools import product

with ProcessPoolExecutor() as executor:
    results = list(executor.map(
        lambda args: validate_training_duration_limitation(*args), 
        product(config.datasets, config.curricula)
    ))
results

PicklingError: Can't pickle <function <lambda> at 0x7fe6e4d0f880>: attribute lookup <lambda> on __main__ failed